<a href="https://colab.research.google.com/github/moonstar0301/2023_1_BackTest/blob/main/%EB%8B%A4%ED%8A%B812%EA%B8%B0%EC%B5%9C%EB%AC%B8%EC%84%9D_BackTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

다트 12기 최문석 백테스트입니다.


제 전략은 특정 매크로 국면에 맞춰져 있습니다.
따라서 '백테스트'를 학습해보는 용도로 제 전략을 그대로 적용하는것은 알맞지 않다고 생각해서 약간의 변화를 준 후에 백테스트를 진행해보았습니다.

<2023년 상반기 모의투자 전략>

https://github.com/moonstar0301/KNUDART_Investing


모의투자에서는 투자 기간이 짧고 현재 거시 경제 상항을 고려해서 보다 안정적인 수익을 낼 수 있는 단기채권 ETF로 Core를 구상했지만 백테스트는 기간을 길게 잡고 전략의 유의미성을 검증하는것으로 이번 백테스트에서는 Core를 시장 지수로 잡고 백테스트를 진행해 보겠습니다.

Satellite는 현재 거시 경제 상황을 고려해서 유니버스를 경기 민감 종목으로 한정했으나 백테스트에서는 유니버스를 코스피로 잡고 퀄리티 팩터와 밸류 팩터를 결합해서 사용하겠습니다.


<팩터 구성>

퀄리티 팩터 : 신 F-score 사용(3점이상인 기업 스크리닝)
-당기순이익이 0 이상인가?
-영업현금흐름이 0 이상인가?
-당해 신규주식 발행을 하지 않았는가?

밸류 팩터 : 
Rank(PER) + Rank(PBR) + Rank(PCR) + Rank(PSR)

*개별 지표의 랭킹을 합 -> 상위 20개 기업 스크리닝

** 20% 이상 하락시 손절하겠습니다.

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
mpl.rc('font', family = 'Malgun Gothic')
import matplotlib.pyplot as plt
import datetime
import math

import plotly.graph_objects as go # 그래프 그릴 때 사용할 라이브러리입니다.
from plotly.subplots import make_subplots

!pip install pykrx
from pykrx import stock # 한국 거래소 API로 데이터 받아올 수 있는 라이브러리입니다.
from pykrx import bond

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.5/125.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 30.1 MB/s eta 0:00:00


In [2]:
# 리밸런싱 조건 설정해주기
# 리밸런싱 주기
rebalancing = 91 #일 단위

# 거래비용
tax = 0.0002 # 현재 거래비용은 0.02%입니다.

# 투자원금
cash = 100000000 # 모의투자 시작금액인 1억으로 시작하겠습니다.
money = cash

risk_free_rate = 0.0355 / 365 # 23년 5월 4일 기준 CD91일물 금리로 하겠습니다.

# 손절기준
loss_cut = 0.2 #매수 이후 20% 이상 가격이 떨어지면 매도하겠습니다.

day = 0
quarter = 0
df_list = '현금 주식총액 포트폴리오가치 일일수익률 총수익률'.split() # 백테스트 데이터프레임에 보여줄 항목들입니다. 이후에 더 추가할거에요
backtest = pd.DataFrame(columns = df_list)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. 데이터 불러오기

In [4]:
kospi = pd.read_excel('/content/drive/MyDrive/수정주가.xlsx', index_col = 0, sheet_name='KOSPI')
kosdaq = pd.read_excel('/content/drive/MyDrive/수정주가.xlsx', index_col = 0, sheet_name='KOSDAQ')
price_data = pd.concat([kospi, kosdaq], axis = 1) # 코스피 주가와 코스닥 주가를 합쳐줍니다.

PER_KOSPI = pd.read_excel('/content/drive/MyDrive/신_fscore.xlsx', index_col = 0, sheet_name = 'PER')
PBR_KOSPI = pd.read_excel('/content/drive/MyDrive/신_fscore.xlsx', index_col = 0, sheet_name = 'PBR')
PCR_KOSPI = pd.read_excel('/content/drive/MyDrive/신_fscore.xlsx', index_col = 0, sheet_name = 'PCR')
PSR_KOSPI = pd.read_excel('/content/drive/MyDrive/신_fscore.xlsx', index_col = 0, sheet_name = 'PSR')

In [5]:
price_data

,A005930,A373220,A000660,A207940,A051910,A006400,A005380,A035420,A000270,A005490,...,A900020,A900040,A900060,A900080,A900090,A900130,A900150,A900180,A950030,A950180
Symbol,,,,,,,,,,,,,,,,,,,,,
2012-01-01,21160,NaN,21950,NaN,317500,133500,213000,66234,66700,380000,...,NaN,1563.0,2640.0,1543.0,2084.0,509.0,1660.0,1961.0,NaN,NaN
2012-01-02,21600,NaN,23450,NaN,320000,137500,212500,65606,66500,376000,...,NaN,1536.0,2630.0,1547.0,2048.0,510.0,1620.0,1898.0,NaN,NaN
2012-01-03,22100,NaN,23650,NaN,330500,141000,221500,68902,68100,387500,...,NaN,1527.0,2660.0,1595.0,2106.0,500.0,1650.0,1889.0,NaN,NaN
2012-01-04,21600,NaN,23850,NaN,324500,138500,223500,68902,68700,387500,...,NaN,1554.0,2740.0,1602.0,2099.0,490.0,1650.0,1838.0,NaN,NaN
2012-01-05,21100,NaN,24000,NaN,331500,134000,223500,67960,68400,392000,...,NaN,1554.0,2715.0,1606.0,2113.0,494.0,1750.0,1840.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-26,53900,446500.0,82500,758000.0,571000,597000,182500,204500,74800,224500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-27,54200,444000.0,82000,754000.0,569000,587000,186000,204500,76500,229000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-28,52900,433500.0,81200,767000.0,546000,564000,179500,200500,73900,228000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Excel 파일 '스크리닝' 구조

1. 분기별 영업활동 현금흐름(CFO)
2. 분기별 당기 순이익(NetIncome)
3. 분기별 신규주식 발행수(Issuance of new shares)
-> 신 f-score

4. 분기별 PER
5. 분기별 PBR
6. 분기별 PCR
7. 분기별 PSR
-> Rank(PER) + Rank(PBR) + Rank(PCR) + Rank(PSR)

In [6]:
# 신 f-score
CFO_KOSPI = pd.read_excel('/content/drive/MyDrive/My_screening.xlsx', index_col = 0, sheet_name = '당기순이익')
NI_KOSPI = pd.read_excel('/content/drive/MyDrive/My_screening.xlsx', index_col = 0, sheet_name = '영업활동으로인한 현금흐름')
INS_KOSPI = pd.read_excel('/content/drive/MyDrive/My_screening.xlsx', index_col = 0, sheet_name = '상장예정주식')

"""
# BLEND_RANK
PER_KOSPI = pd.read_excel('스크리닝.xlsx', index_col = 0, sheet_name = 'PER_KOSPI')
PBR_KOSPI = pd.read_excel('스크리닝.xlsx', index_col = 0, sheet_name = 'PBR_KOSPI')
PCR_KOSPI = pd.read_excel('스크리닝.xlsx', index_col = 0, sheet_name = 'PCR_KOSPI')
PSR_KOSPI = pd.read_excel('스크리닝.xlsx', index_col = 0, sheet_name = 'PSR_KOSPI')
"""

"\n# BLEND_RANK\nPER_KOSPI = pd.read_excel('스크리닝.xlsx', index_col = 0, sheet_name = 'PER_KOSPI')\nPBR_KOSPI = pd.read_excel('스크리닝.xlsx', index_col = 0, sheet_name = 'PBR_KOSPI')\nPCR_KOSPI = pd.read_excel('스크리닝.xlsx', index_col = 0, sheet_name = 'PCR_KOSPI')\nPSR_KOSPI = pd.read_excel('스크리닝.xlsx', index_col = 0, sheet_name = 'PSR_KOSPI')\n"

In [7]:
CFO_KOSPI = CFO_KOSPI.transpose()
NI_KOSPI = NI_KOSPI.transpose()
INS_KOSPI = INS_KOSPI.transpose()
CFO_KOSPI

item,Protfolio,A005930,A373220,A000660,A207940,A051910,A006400,A005380,A000270,A005490,...,A435870,A418210,A413600,A373340,A442130,A438580,A400560,A405350,A050540,A304360
2012,1Q,3913284000000,NaN,-298653000000,NaN,310361000000,14026397000,1416636000000,978686000000,538559370000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,857161000,NaN
2012.1,2Q,7621608000000,NaN,-375291000000,NaN,751330000000,27538548000,3305204000000,1551653000000,1242522075000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1277675000,NaN
2012.2,3Q,12452412000000,NaN,-538440000000,NaN,1163113000000,2590852164000,4051116000000,1798040000000,1986617838000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1955114000,NaN
2012.3,4Q,17398530000000,NaN,-489990000000,-34950449860,1440317000000,2547189629140,5279509000000,2136797000000,2499522911070,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2095948040,NaN
2013,1Q,4211422000000,NaN,83851000000,NaN,307201000000,-8835105000,1326618000000,1038454000000,383424959000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,387498000,NaN
2013.1,2Q,9333829000000,NaN,1018886000000,NaN,726975000000,53513115000,3050178000000,1730195000000,765042537000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117214000,NaN
2013.2,3Q,13961693000000,NaN,2131937000000,NaN,1009524000000,128559384000,3924090000000,2071396000000,1380110995000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,910525000,NaN
2013.3,4Q,17929520000000,NaN,2796967000000,-62418551750,1174392000000,-63362664050,5181546000000,2582486000000,1582595860510,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9967767960,NaN
2014,1Q,4968366000000,NaN,875358000000,NaN,254398000000,-5738056840,1402543000000,1011040000000,111519121000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,442003000,NaN
2014.1,2Q,9419000000000,NaN,1527245000000,NaN,473248000000,54374817000,3544808000000,1928643000000,658754438000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,766880000,NaN


In [8]:
CFO_KOSPI.columns
CFO_KOSPI.index

Index([    2012, '2012.1', '2012.2', '2012.3',     2013, '2013.1', '2013.2',
       '2013.3',     2014, '2014.1', '2014.2', '2014.3',     2015, '2015.1',
       '2015.2', '2015.3',     2016, '2016.1', '2016.2', '2016.3',     2017,
       '2017.1', '2017.2', '2017.3',     2018, '2018.1', '2018.2', '2018.3',
           2019, '2019.1', '2019.2', '2019.3',     2020, '2020.1', '2020.2',
       '2020.3',     2021, '2021.1', '2021.2', '2021.3',     2022, '2022.1',
       '2022.2', '2022.3'],
      dtype='object')

In [9]:
#CFO_KOSPI
#NI_KOSPI
#INS_KOSPI
CFO_KOSPI = CFO_KOSPI.iloc[:, 1:] # 0번째 column을 삭제하고 나머지만 남긴다.

In [10]:
NI_KOSPI = CFO_KOSPI.iloc[:, 1:]
INS_KOSPI = CFO_KOSPI.iloc[:, 1:]

In [11]:
INS_KOSPI

item,A373220,A000660,A207940,A051910,A006400,A005380,A000270,A005490,A035420,A003670,...,A435870,A418210,A413600,A373340,A442130,A438580,A400560,A405350,A050540,A304360
2012,NaN,-298653000000,NaN,310361000000,14026397000,1416636000000,978686000000,538559370000,131811957000,18824228000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,857161000,NaN
2012.1,NaN,-375291000000,NaN,751330000000,27538548000,3305204000000,1551653000000,1242522075000,259016302000,40586919000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1277675000,NaN
2012.2,NaN,-538440000000,NaN,1163113000000,2590852164000,4051116000000,1798040000000,1986617838000,384616317000,60028350000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1955114000,NaN
2012.3,NaN,-489990000000,-34950449860,1440317000000,2547189629140,5279509000000,2136797000000,2499522911070,536116221190,78553652850,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2095948040,NaN
2013,NaN,83851000000,NaN,307201000000,-8835105000,1326618000000,1038454000000,383424959000,153517114000,11286276000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,387498000,NaN
2013.1,NaN,1018886000000,NaN,726975000000,53513115000,3050178000000,1730195000000,765042537000,269294937000,27396114000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117214000,NaN
2013.2,NaN,2131937000000,NaN,1009524000000,128559384000,3924090000000,2071396000000,1380110995000,1861071077000,44652256000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,910525000,NaN
2013.3,NaN,2796967000000,-62418551750,1174392000000,-63362664050,5181546000000,2582486000000,1582595860510,1960643768140,59953187060,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9967767960,NaN
2014,NaN,875358000000,NaN,254398000000,-5738056840,1402543000000,1011040000000,111519121000,124898265000,18446858000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,442003000,NaN
2014.1,NaN,1527245000000,NaN,473248000000,54374817000,3544808000000,1928643000000,658754438000,171515934000,37190843000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,766880000,NaN


In [12]:
NI_KOSPI

item,A373220,A000660,A207940,A051910,A006400,A005380,A000270,A005490,A035420,A003670,...,A435870,A418210,A413600,A373340,A442130,A438580,A400560,A405350,A050540,A304360
2012,NaN,-298653000000,NaN,310361000000,14026397000,1416636000000,978686000000,538559370000,131811957000,18824228000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,857161000,NaN
2012.1,NaN,-375291000000,NaN,751330000000,27538548000,3305204000000,1551653000000,1242522075000,259016302000,40586919000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1277675000,NaN
2012.2,NaN,-538440000000,NaN,1163113000000,2590852164000,4051116000000,1798040000000,1986617838000,384616317000,60028350000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1955114000,NaN
2012.3,NaN,-489990000000,-34950449860,1440317000000,2547189629140,5279509000000,2136797000000,2499522911070,536116221190,78553652850,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2095948040,NaN
2013,NaN,83851000000,NaN,307201000000,-8835105000,1326618000000,1038454000000,383424959000,153517114000,11286276000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,387498000,NaN
2013.1,NaN,1018886000000,NaN,726975000000,53513115000,3050178000000,1730195000000,765042537000,269294937000,27396114000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117214000,NaN
2013.2,NaN,2131937000000,NaN,1009524000000,128559384000,3924090000000,2071396000000,1380110995000,1861071077000,44652256000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,910525000,NaN
2013.3,NaN,2796967000000,-62418551750,1174392000000,-63362664050,5181546000000,2582486000000,1582595860510,1960643768140,59953187060,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9967767960,NaN
2014,NaN,875358000000,NaN,254398000000,-5738056840,1402543000000,1011040000000,111519121000,124898265000,18446858000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,442003000,NaN
2014.1,NaN,1527245000000,NaN,473248000000,54374817000,3544808000000,1928643000000,658754438000,171515934000,37190843000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,766880000,NaN


In [23]:
#CFO_KOSPI
#NI_KOSPI
#INS_KOSPI

print('전체 리밸런싱 횟수는 {}'.format(int(len(price_data)/rebalancing)))
for reb in range(int(len(price_data)/rebalancing)): #리밸런싱 횟수를 계산해줍니다. 전체 투자기간/리밸런싱 주기
    print( reb+1, '회 투자')
    print('현금 : ', money)
    inv = price_data.iloc[day:day+rebalancing,:] #전체 주가 데이터에서 투자기간동안의 데이터를 뽑아냅니다.
    inv = inv.replace(np.NaN, 0) # 수정주가 데이터에 없는 값들은 0으로 처리해줍니다.
    inv = inv.loc[:,inv.iloc[0,:] > 0 ] # 투자 시작 시점에 데이터가 0인 친구들은 다 날려줍니다.
    # 스크리닝


    CFO_KOSPI_inv = pd.DataFrame(CFO_KOSPI.iloc[quarter, :]) # 투자 시점의 데이터를 불러와줍니다.
    CFO_KOSPI_inv = CFO_KOSPI_inv.dropna(axis = 'index') # 투자 시점에 없는 데이터들은 제거해줍니다.
    CFO_KOSPI_inv = CFO_KOSPI_inv.replace(np.NaN, 0) # 수정주가 데이터에 없는 값들은 0으로 처리해줍니다.
    #CFO_KOSPI_inv = CFO_KOSPI_inv.sort_values(by = CFO_KOSPI_inv.columns[0], ascending = False) # 내림차순으로 정렬해주고
    CFO_KOSPI_inv.astype(int, copy=False)
    CFO_KOSPI_inv = CFO_KOSPI_inv[CFO_KOSPI_inv[CFO_KOSPI_inv.columns[0]] > 0] # CFO가 0 이상인 기업들만 남깁니다.
    CFO_KOSPI_inv_list = CFO_KOSPI_inv.index # 리스트로 저장해줍니다.

    NI_COSPI_inv = pd.DataFrame(NI_KOSPI.iloc[quarter, :]) # 투자 시점의 데이터를 불러와줍니다.
    NI_COSPI_inv = NI_COSPI_inv.dropna(axis = 'index') # 투자 시점에 없는 데이터들은 제거해줍니다.
    #NI_COSPI_inv = NI_COSPI_inv.sort_values(by = NI_COSPI_inv.columns[0], ascending = False) # 내림차순으로 정렬해주고
    NI_COSPI_inv = NI_COSPI_inv[NI_COSPI_inv[NI_COSPI_inv.columns[0]] > 0] # NI가가 0 이상인 기업들만 남깁니다.
    NI_COSPI_inv_list = NI_COSPI_inv.index # 리스트로 저장해줍니다.



    INS_KOSPI_inv = pd.DataFrame(INS_KOSPI.iloc[quarter, :])
    INS_KOSPI_inv.dropna(axis = 'index', inplace = True) # NaN값 제거해주고
    #INS_KOSPI_inv = INS_KOSPI_inv.sort_values(by = INS_KOSPI_inv.columns[0], ascending=False) # 내림차순으로 정렬하겠습니다.
    INS_KOSPI_inv = INS_KOSPI_inv[INS_KOSPI_inv.iloc[:,0] > 0] # INS가 0인 기업만 추려줍니다.
    INS_KOSPI_inv_list = INS_KOSPI_inv.index #리스트로 저장해줍니다.


    constraint = CFO_KOSPI_inv_list.intersection(NI_COSPI_inv_list) # 두 조건 모두에 걸리는 종목들을 선정해줍니다
    constraint = constraint.intersection(INS_KOSPI_inv_list)
#___________________________________________________________________________________________________________________________________
    #inv_list = constraint
    #밸류팩터 쓰면 여기서 줄세워서 상위 20개만 뽑아야함함
    PER_inv = pd.DataFrame(PER_KOSPI.iloc[quarter, :]) # 투자 시점의 데이터를 불러와줍니다.
    PER_inv = PER_inv.transpose()
    PER_inv = PER_inv.loc[:,constraint]
    PBR_inv = pd.DataFrame(PER_KOSPI.iloc[quarter, :]) # 투자 시점의 데이터를 불러와줍니다.
    PBR_inv = PBR_inv.transpose()
    PBR_inv = PBR_inv.loc[:,constraint]
    PCR_inv = pd.DataFrame(PER_KOSPI.iloc[quarter, :]) # 투자 시점의 데이터를 불러와줍니다.
    PCR_inv = PCR_inv.transpose()
    PCR_inv = PCR_inv.loc[:,constraint]
    PSR_inv = pd.DataFrame(PER_KOSPI.iloc[quarter, :]) # 투자 시점의 데이터를 불러와줍니다.
    PSR_inv = PSR_inv.transpose()
    PSR_inv = PSR_inv.loc[:,constraint]

    PER_inv = PER_inv.transpose()
    PBR_inv = PBR_inv.transpose()
    PCR_inv = PCR_inv.transpose()
    PSR_inv = PSR_inv.transpose()

    BLEND = pd.DataFrame(index = constraint)
    BLEND['PER'] = PER_inv
    BLEND['PBR'] = PBR_inv
    BLEND['PCR'] = PCR_inv
    BLEND['PSR'] = PSR_inv

    BLEND["PCR_RANK"] = BLEND["PCR"].rank(ascending=False)
    BLEND["PER_RANK"] = BLEND["PER"].rank(ascending=False)
    BLEND["PBR_RANK"] = BLEND["PBR"].rank(ascending=False)
    BLEND["PSR_RANK"] = BLEND["PSR"].rank(ascending=False)

    BLEND['total_rank'] = BLEND["PCR_RANK"] + BLEND["PBR_RANK"] +BLEND["PSR_RANK"] + BLEND["PER_RANK"]

    result = BLEND.sort_values("total_rank", ascending = True)
    inv_list = result[0:20].index




    print('투자 후보 갯수는 : ', len(inv_list))




#___________________________________________________________________________________________________________________________________
    #여기까지 스크리닝 과정입니다.
    final_inv_list = []
    for i in range(len(inv_list)):
        if inv_list[i] in inv.columns:
            final_inv_list.append(inv_list[i])
        else:
            print(inv_list[i],' 종목이 없습니다')
    print('투자하는 종목의 수는 : ', len(final_inv_list))
    print('투자종목 : ',final_inv_list)
    # 스크리닝 된 종목 들 중 가격 데이터에 있는 종목들만 투자 리스트에 추가해줍니다.


    # 다음은 보유한 주식의 가치를 계산하기 위한 과정입니다.

    if len(final_inv_list)==0:
        allocation=0
    else:
        allocation = money / len(final_inv_list) # 동일 비중이기 때문에 보유 현금 / 투자할 종목 수 로 나눠줍니다.

    
    print('동일 비중 투자 금액은 : ' , allocation) # 저는 제 코드를 믿지 못하기 때문에 이렇게 중간중간 어디서 에러가 나는지 예상할 수 있게 출력 결과도 함께 프린트합니다.
    final_price_data = inv[final_inv_list].copy()



    vec = pd.DataFrame({'매수수량' : allocation // final_price_data.iloc[0,:]}) # 종목당 매수할 수를 데이터프레임(벡터) 로 만들어줍니다.
    vec = vec.replace(np.NaN, 0)

    # 손절을 할 때 보유하고 있는 주식수를 0으로 만들거나
    # 손절 이후 포트폴리오의 가격을 0으로 만들거나의 방법을 선택할 수 있겠죠?
    # 저는 손절 한 순간부터 포트폴리오에서 주가를 계산하지 않아도 되는 방법이 더 편할 것 같아서 이렇게 선택했지만
    # 저와 다르게 계산하셔도 상관없습니다.
    loss_cut_money_list = []
    loss_cut_money = 0
    for days in final_price_data.index:
        for stocks in final_price_data.columns:
            if final_price_data.loc[days, stocks] < final_price_data[stocks][0] * ( 1 - loss_cut ):
                loss_cut_money = loss_cut_money + (final_price_data.loc[days, stocks] * float(vec.loc[stocks])) * (1 - tax)
                final_price_data.loc[days:, stocks] = 0
        
        loss_cut_money_list.append(loss_cut_money)


        # 날짜에 저장된 값을 리스트에 추가해줍니다. 이 반복문을 돌면서 포트폴리오 투자기간동안의 현금이 저장될 것입니다.

   

    product = np.dot(final_price_data, vec) #np.dot 통해 행렬과 벡터의 내적값을 구해줍니다.
    product = pd.DataFrame(product) #데이터프레임으로 만들어서 나중에 붙여주겠습니다.



    balance = pd.DataFrame(index = product.index) 
    balance['현금'] = money - product.iloc[0,0] # 현금은 리밸런싱이 이루어지는 날의 현금에서 주식 매수대금을 뺀 값입니다.
    balance['현금'] += loss_cut_money_list # 손절한 금액은 날짜별로 더해줍시다

    
    _backtest = pd.DataFrame(columns = df_list) # 빈 데이터프레임을 만들어줍니다.
    _backtest['주식총액'] = product # 보유하고 있는 주식의 가치입니다.
    _backtest['현금'] = balance['현금'] # 아까 계산한 일자별 현금 데이터를 넣어줍니다.


    _backtest['포트폴리오가치'] = _backtest['현금'] + _backtest['주식총액'] # 포트폴리오 가치는 현금과 주식총액의 합입니다.
    _backtest['총수익률'] = (_backtest['포트폴리오가치']/cash) # 처음 들고 시작한 돈 대비 수익률
    _backtest.index = price_data.index[day : day + rebalancing]


    backtest = pd.concat([backtest, _backtest], axis = 0, ignore_index=False) 
    # 데이터프레임들을 계속 붙여주면서 백테스트 데이터를 만들겠습니다. 
    # 반복문을 돌면서 빈데이터프레임에 데이터를 채워가기 때문에 먼저 선언해준 백테스트 데이터프레임에 붙여줍니다.


    # 리밸런싱 이후 시작 금액은 주식 판매대금 - 거래비용 + 현금입니다.
    money = backtest.iloc[-1,0] + (backtest.iloc[-1,1] * ( 1 - tax))
    day = day + rebalancing
    quarter += 1 

backtest['일일수익률'] = backtest['포트폴리오가치'].pct_change()


전체 리밸런싱 횟수는 43
1 회 투자
현금 :  100000000
투자 후보 갯수는 :  20
A140410  종목이 없습니다
투자하는 종목의 수는 :  19
투자종목 :  ['A086060', 'A010770', 'A014910', 'A032280', 'A035000', 'A006730', 'A082210', 'A092070', 'A058530', 'A067290', 'A003000', 'A023900', 'A085670', 'A024810', 'A043260', 'A037370', 'A101680', 'A006570', 'A046390']
동일 비중 투자 금액은 :  5263157.894736842
2 회 투자
현금 :  121238455.84040001
투자 후보 갯수는 :  20
투자하는 종목의 수는 :  20
투자종목 :  ['A060310', 'A039560', 'A030350', 'A078160', 'A007570', 'A006730', 'A032620', 'A041960', 'A090360', 'A088910', 'A011070', 'A027040', 'A101670', 'A002820', 'A019570', 'A020150', 'A007110', 'A011420', 'A036570', 'A004140']
동일 비중 투자 금액은 :  6061922.7920200005
3 회 투자
현금 :  105458218.853
투자 후보 갯수는 :  20
A104540  종목이 없습니다
투자하는 종목의 수는 :  19
투자종목 :  ['A006730', 'A104200', 'A022100', 'A008040', 'A060310', 'A041190', 'A030960', 'A032800', 'A900100', 'A003520', 'A030350', 'A007570', 'A048910', 'A014820', 'A041960', 'A032850', 'A127120', 'A085310', 'A072710']
동일 비중 투자 금액은 :  5550432.5712105

In [24]:
type(constraint)

pandas.core.indexes.base.Index

In [25]:
BM = pd.DataFrame(stock.get_index_ohlcv(backtest.index[0], backtest.index[-1], "1001", "d")['시가']) # 코스피 지수를 벤치마크로 설정
BM['일일수익률'] = BM['시가'].pct_change()
BM['총수익률'] = BM['시가']/BM['시가'][0] 
BM

,시가,일일수익률,총수익률
날짜,,,
2012-01-02,1831.69,NaN,1.000000
2012-01-03,1846.56,0.008118,1.008118
2012-01-04,1883.48,0.019994,1.028274
2012-01-05,1869.42,-0.007465,1.020598
2012-01-06,1867.12,-0.001230,1.019343
...,...,...,...
2022-09-08,2393.27,-0.000998,1.306591
2022-09-13,2418.59,0.010580,1.320414
2022-09-14,2390.47,-0.011627,1.305063


In [26]:
# CAGR : 연평균 성장률 ( Compound Annual Growth Rate)
num_of_year = int(len(backtest)/365)

CAGR = (backtest['포트폴리오가치'][-1]/cash)**(1/num_of_year) - 1
print('포트폴리오 CAGR은 {} %'.format(CAGR*100))

BENCHMARK_CAGR = ((BM['시가'][-1] / BM['시가'][0])) ** (1 / num_of_year) - 1
print("벤치마크 CAGR은 {} %".format(BENCHMARK_CAGR*100))

포트폴리오 CAGR은 30.368859349588263 %
벤치마크 CAGR은 2.6857624178796558 %


In [27]:
#MDD : 최저점이 최고점 대비 몇퍼센트의 하락인지?
# (최고점 - 최저점) / 최고점

max_list = [backtest.iloc[0,2]]
min_list = [backtest.iloc[0,2]]

for i in range(len(backtest)):
    if i==0:
        max_list.append(backtest.iloc[0,2])
        min_list.append(backtest.iloc[0,2])

    else:   
        if backtest.iloc[i, 2] >= backtest.iloc[i-1,2]:
            max_list.append(backtest.iloc[i, 2])
            min_list.append(backtest.iloc[i, 2])
        else:
            max_list.append(max_list[-1])
            min_list.append(backtest.iloc[i, 2])


max_list = max_list[1:]
min_list = min_list[1:]
backtest['max'] = max_list
backtest['min'] = min_list
backtest['mdd'] = -((backtest['max'] - backtest['min'])/backtest['max'])

print('BACKTEST MDD는 {}%'.format(backtest['mdd'].min()*100))

backtest['MDD'] = 0
for i in range(len(backtest)):
    if i != 0:
        backtest.iloc[i, -1] = backtest['mdd'][:i].min()

    

BACKTEST MDD는 -15.02823549600546%


In [28]:
BENCHMARK_max_list = [BM.iloc[0,0]]
BENCHMARK_min_list = [BM.iloc[0,0]]

for i in range(len(BM)):
    if i==0:
        BENCHMARK_max_list.append(BM.iloc[0,0])
        BENCHMARK_min_list.append(BM.iloc[0,0])

    else:   
        if BM.iloc[i, 2] >= BM.iloc[i-1,2]:
            BENCHMARK_max_list.append(BM.iloc[i, 0])
            BENCHMARK_min_list.append(BM.iloc[i, 0])
        else:
            BENCHMARK_max_list.append(BENCHMARK_max_list[-1])
            BENCHMARK_min_list.append(BM.iloc[i, 0])


BENCHMARK_max_list = BENCHMARK_max_list[1:]
BENCHMARK_min_list = BENCHMARK_min_list[1:]
BM['max'] = BENCHMARK_max_list
BM['min'] = BENCHMARK_min_list
BM['mdd'] = -((BM['max'] - BM['min'])/BM['max'])

print('BENCHMARK MDD는 {}%'.format(BM['mdd'].min()*100))

BM['MDD'] = 0
for i in range(len(BM)):
    if i != 0:
        BM.iloc[i, -1] = BM['mdd'][:i].min()

BENCHMARK MDD는 -12.553673522643694%


In [29]:
# Sharpe Ratio = (자산 X의 기대수익률 – 무위험 자산 수익률) / 자산 X의 기대수익률의 표준편차
backtest_return = backtest['일일수익률'].mean()
backtest_std = backtest['일일수익률'].std()

BM_return = BM['일일수익률'].mean()
BM_std = BM['일일수익률'].std()

backtest_sharpe = (backtest_return - risk_free_rate) / backtest_std
BM_sharpe = (BM_return - risk_free_rate) / BM_std

print('BACKTEST 샤프 비율은 : ', backtest_sharpe)
print('BENCHMARK 샤프 비율은 : ', BM_sharpe)

BACKTEST 샤프 비율은 :  0.05497960846603631
BENCHMARK 샤프 비율은 :  0.005311753981854069


In [30]:
fig = make_subplots(rows=4, cols=1,
                    specs=[[{"rowspan":3}],
                          [None],
                          [None],
                          [{}]],
                   shared_xaxes=True,
                   vertical_spacing=0.2,
                   subplot_titles=("수익률","MDD"))

fig.add_trace(go.Scatter(name='코스피 총수익률', x=BM.index, y=BM['총수익률']),
             row=1, col=1)
fig.add_trace(go.Scatter(name='포트폴리오 총수익률', x=backtest.index, y=backtest['총수익률']),
             row=1, col=1)
fig.add_trace(go.Scatter(name='포트폴리오 MDD', x=backtest.index, y=backtest['MDD'], fill='tozeroy'),
             row=4, col=1)
fig.add_trace(go.Scatter(name='BENCHMARK MDD', x=BM.index, y=BM['MDD'], fill='tozeroy'),
             row=4, col=1)

fig.update_layout(height=800, width=1000, plot_bgcolor='rgb(240, 240,240)',
                 title_text="백테스트 결과")

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()